In [ ]:
#Importing required dependencies
import os    
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [1]:
!unzip "/content/drive/MyDrive/Mask_detection/images.zip" -d"/content/drive/MyDrive/Mask_detection"  #Unzip all the images in to Mask_detection folder

In [ ]:
%cd "/content/drive/MyDrive/Mask_detection/images"

/content/drive/MyDrive/Mask_detection/images


In [2]:
!git clone "https://github.com/alexeyAB/darknet" "/content/drive/MyDrive/Mask_detection/darknet"  #Clonning darknet 

In [3]:
%cd /content/drive/MyDrive/Mask_detection/darknet

In [4]:
!make #Compilation process

In [5]:
%cd /content/drive/MyDrive/Mask_detection  

In [ ]:
!python mask_detection_folder/creating-files-data-and-name.py  

In [ ]:
!python mask_detection_folder/creating-train-and-test-txt-files.py  #Divides the images into train and test images

In [6]:
!darknet/darknet detector train mask_detection_folder/labelled_data.data darknet/cfg/yolov3_mask.cfg mask_weights/darknet53.conv.74 -dont_show #training the model

In [7]:
#Testing part

net = cv2.dnn.readNetFromDarknet("/content/drive/MyDrive/Mask_detection/darknet/cfg/yolov3_mask.cfg",r"/content/drive/MyDrive/Mask_detection/backup/yolov3_mask_last.weights") #Reading model and configuration file

classes = ["without mask","with mask"]


image_path = ["/content/drive/MyDrive/Mask_detection/mask_detection_folder/0.jpeg","/content/drive/MyDrive/Mask_detection/mask_detection_folder/36.jpeg","/content/drive/MyDrive/Mask_detection/mask_detection_folder/50.jpeg"]
for u in image_path:
  image = cv2.imread(u)
  height , width, _ = image.shape

  blob = cv2.dnn.blobFromImage(image , 1/255 , (416,416) ,(0,0,0) , swapRB = True , crop = False) #converting an image into specific form which can be feeded to model

  net.setInput(blob)  #Input
  output_layer_names = net.getUnconnectedOutLayersNames()

  layeroutputs = net.forward(output_layer_names)

  #Extracting details of bounding boxes, confidence and classes of the image
  boxes=[]
  confidences=[]
  class_id=[]
  
  for output in layeroutputs:
      
      for detection in output:
          score = detection[5:]
          ids = np.argmax(score)
          confidence = score[ids]
          
          if confidence > 0.5: #setting confidence thresold
              center_x = int(detection[0]*width)
              center_y = int(detection[1]*height)
              w = int(detection[2]*width)
              h = int(detection[3]*height)
            
              x = int(center_x - w/2)
              y = int(center_y - h/2)
            
              boxes.append([x,y,w,h])
              confidences.append(float(confidence))
              class_id.append(ids)

  indexes = cv2.dnn.NMSBoxes(boxes,confidences ,0.5,0.4) #Picking the boxes having high confidence

  #Plotting the bounding boxes
  font = cv2.FONT_HERSHEY_PLAIN
  colors = np.random.uniform(0,255,size=(len(boxes), 3))
  if len(indexes)>0:
      for i in indexes.flatten():
          x,y,w,h = boxes[i]
          label = str(classes[class_id[i]])
          confi = str(round(confidences[i],2))
          color = colors[i]
          cv2.rectangle(image , (x,y) , (x+w,y+h) , color ,3)
          cv2.putText(image , label +" "+confi , (x,y) ,font ,3 ,(255,255,255),2)
  
  plt.figure(figsize=(5,5))
  cv2_imshow(image)
  cv2.imwrite('output.jpg',image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()